In [1]:
from google.colab import files
import json
import requests
from datetime import datetime
import pandas as pd

## Configs upload

In [2]:
uploaded = files.upload()

Saving config.txt to config.txt


In [3]:
uploaded_string = str(uploaded)
configs = json.loads(uploaded_string[uploaded_string.find('b')+2: uploaded_string.rfind('\'')])

## Users upload

In [4]:
users = files.upload()

Saving users.csv to users.csv


In [5]:
users_string = str(users)
users = users_string[users_string.find('b')+2:users_string.rfind('\'')]

In [6]:
users_list = users.split('\\n')

del users_list[0] 
del users_list[-1] 

## Data download

In [7]:
def smile_request(userId, env='test', method='employments', pagination=10):

    methods = ['employments', 'incomes', 'insights', 'contributions', 'identities', 'ratings', 'transactions', 'liabilities']

    if env == 'test':
        config = configs['config_test']
    elif env == 'prod':
        config = configs['config_prod']
    else:
        raise Exception('Incorrect {env} env value'.format(env=env))

    if type(pagination) != int:
        raise Exception('Incorrect pagination data type. Must be integer instead of {datatype}'.format(datatype=type(pagination)))
    
    if method not in methods:
      raise Exception('Method value must be from list')

    url = config['base_url'] + method + '?size=' + str(pagination) + '&userId=' + userId
    headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": config['auth_key']
    }
    payload={}

    response = requests.request("GET", url, headers=headers, data=payload)
    return response

In [14]:
def batch_request(users, env='test', method='employments', pagination=10):

      responses = []
      k = 0
      # f = open("logs_{method}_{datetime}.json".format(method=method, datetime=str(datetime.now())), "w")
      file_name = 'logs_' + method + '_' + str(datetime.now()) + '.json'

      for user in users:
        response = smile_request(user, env=env, method=method, pagination=pagination)
        # print(response)
        data_json = json.loads(response.text)
        data = data_json['data']['items']

        
        with open(file_name, 'a') as outfile:
          json_string = json.dumps(data)
          outfile.write(json_string)
        k += 1
        for i in data:
          responses.append(i)
        if k % 50 == 0:
          print('{k} users was processed'.format(k=k))

      return responses

In [23]:
batch_resp = batch_request(users_list, env='prod', method='liabilities')

50 users was processed
100 users was processed
150 users was processed
200 users was processed
250 users was processed
300 users was processed
350 users was processed
400 users was processed
450 users was processed
500 users was processed
550 users was processed
600 users was processed
650 users was processed
700 users was processed


In [24]:
with open('json_data_liabilities.json', 'w') as outfile:
      json_string = json.dumps(batch_resp)
      outfile.write(json_string)